<span style="color: purple">

Load in stored variables:

</span>

In [6]:
%store -r data_dir sites_gdf rrs_forest_gdf lp_forest_gdf

<span style="color: purple">

Import packages:

</span>

In [8]:
# Import necessary packages
import os
from math import floor, ceil

import pandas as pd
import matplotlib.pyplot as plt # Overlay pandas and xarray plots
import rioxarray as rxr # Work with raster data
from rioxarray.merge import merge_arrays # Merge rasters

## STEP 2: DATA ACCESS

### Soil data

The [POLARIS dataset](http://hydrology.cee.duke.edu/POLARIS/) is a
convenient way to uniformly access a variety of soil parameters such as
pH and percent clay in the US. It is available for a range of depths (in
cm) and split into 1x1 degree tiles.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download POLARIS data for a particular location, soil parameter,
and soil depth. Your function should account for the situation where
your site boundary crosses over multiple tiles, and merge the necessary
data together.</p>
<p>Then, use loops to download and organize the rasters you will need to
complete this section. Include soil parameters that will help you to
answer your scientific question. We recommend using a soil depth that
best corresponds with the rooting depth of your species.</p></div></div>

<span style="color: purple">

I will focus on soil pH. According to the [USDA PLANTS Database](https://plants.usda.gov/plant-profile/SESE3/characteristics), *S. sempervirens* prefer a soil pH of 5.0-7.0. Also, their minimum root depth is 40 inches (~102 cm), so I will focus on soil depths of 100-200 cm.

</span>

In [3]:
#  Define the URL template to download soil data from POLARIS
def soil_url_temp(variable='variable',
                  statistic='statistic',
                  depth='depth'):
    """
    Create a URL template for downloading soil data from POLARIS.

    Parameters
    ----------
    variable : str
        The soil `variable` of interest.
    statistic : {'mean', 'mode', 'p5', 'p50', 'p95'}
        The `statistic` type of interest.
    depth : {'0_5', '5_15', '15_30', '30_60', '60_100', or '100_200'}
        The `depth` of interest in cm.

    Returns
    -------
    soil_url_template : str
        Template of the URL to download the soil data.
    """
    soil_url_template = ("http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/"
                    "v1.0"
                    f"/{variable}"
                    f"/{statistic}"
                    f"/{depth}"
                    "/lat{min_lat}{max_lat}"
                    "_lon{min_lon}{max_lon}.tif")
    return soil_url_template

In [4]:
# Define site bounds
def define_site_bounds(site_gdf):
    """
    Identify the bounds of a geographic site.
     
    Specifically, this function creates a tuple with
    a DataArray of the site bounds and the minimum longitude,
    minimum latitude, maximum longitude, and maximum latitude of a site.

    Parameters
    ----------
    site_gdf : GeoDataFrame
        The GeoDataFrame of the site of interest.

    Returns
    -------
    site_bounds : DataArray
        A DataArray with the minimum and maximum longitude and
        minimum and maximum latitude of the site.
    site_bounds_min_lon : float
        The minimum longitude of the site.
    site_bounds_min_lat : float
        The minimum latitude of the site.
    site_bounds_max_lon : float
        The maximum longitude of the site.
    site_bounds_max_lat : float
        The maximum latitude of the site.
    """
    site_bounds = (site_bounds_min_lon,
                   site_bounds_min_lat,
                   site_bounds_max_lon,
                   site_bounds_max_lat) = (
                       site_gdf
                       .total_bounds)
    return (site_bounds, site_bounds_min_lon, site_bounds_min_lat,
    site_bounds_max_lon, site_bounds_max_lat)

In [5]:
# Process soil DataArrays
def process_soil_da(site_soil_dir,
                    soil_url_template,
                    site_bounds,
                    site_bounds_min_lon,
                    site_bounds_min_lat,
                    site_bounds_max_lon,
                    site_bounds_max_lat,
                    variable='variable',
                    statistic='statistic',
                    depth='depth'):
    """
    Save, open, scale, and crop raster soil data.

    Parameters
    ----------
    site_soil_dir : str
        The directory where the soil data will be saved.
    soil_url_template : str
        Template of the URL to download the soil data.
    site_bounds : DataArray
        A DataArray with the minimum and maximum longitude and
        minimum and maximum latitude of the site.
    site_bounds_min_lon : float
        The minimum longitude of the site.
    site_bounds_min_lat : float
        The minimum latitude of the site.
    site_bounds_max_lon : float
        The maximum longitude of the site.
    site_bounds_max_lat : float
        The maximum latitude of the site.
    variable : str
        The soil `variable` of interest.
    statistic : {'mean', 'mode', 'p5', 'p50', 'p95'}
        The `statistic` type of interest.
    depth : {'0_5', '5_15', '15_30', '30_60', '60_100', or '100_200'}
        The `depth` of interest in cm.

    Returns
    -------
    site_soil_das : list
        A list of DataArrays of soil data for the
        different rasters covering the site of interest.
    """
    site_soil_das = []
    # List out the soil files for download
    for min_lon in range(floor(site_bounds_min_lon),
                         ceil(site_bounds_max_lon)):
        for min_lat in range(floor(site_bounds_min_lat),
                             ceil(site_bounds_max_lat)):
            max_lon = min_lon + 1
            max_lat = min_lat + 1
            soil_url = soil_url_template.format(
                min_lat=min_lat,
                max_lat=max_lat,
                min_lon=min_lon,
                max_lon=max_lon)
            
            # Define file path for saving downloaded raster data
            soil_path = os.path.join(site_soil_dir,
                                     f"soil_{variable}_{statistic}_{depth}"
                                     f"_lat{min_lat}{max_lat}"
                                     f"_lon{min_lon}{max_lon}.tif")

            if not os.path.exists(soil_path):
                # Save raster data to soil path, mask and scale, squeeze
                soil_da = rxr.open_rasterio(
                    soil_url,
                    mask_and_scale=True
                    ).squeeze()
                soil_da.to_netcdf(soil_path)
                print(f'soil_da saved: {soil_path}')
            else:
                print(f'file already exists: {soil_path}')

            # Open and squeeze raster soil data
            soil_da = rxr.open_rasterio(
                soil_url,
                mask_and_scale=True
            ).squeeze()

            # Crop the raster soil data
            cropped_da = soil_da.rio.clip_box(*site_bounds)
            site_soil_das.append(cropped_da)
            print('Cropped')

    return site_soil_das

In [7]:
# Define data directory for the all data
soil_ph_dir = os.path.join(data_dir, 'soil-ph')

os.makedirs(soil_ph_dir, exist_ok=True)

soil_ph_pattern = os.path.join (soil_ph_dir, '*.tif')
soil_ph_pattern

'C:\\Users\\riede\\earth-analytics\\data\\spring-2025-habitat-suitability\\soil-ph\\*.tif'

In [ ]:
# Loop through bounds of RRS and LP National Forests

# c_s_grasslands_gdf = grasslands_gdf[grasslands_gdf['GRASSLANDN'].isin(['Sheyenne National Grassland', 'Caddo National Grassland'])]
# c_s_grasslands_gdf = [caddo_grasslands_gdf, sheyenne_grasslands_gdf]
# c_s_grasslands_gdf

bounds = []
soil_urls = []
for gdf in sites_gdf:
    bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
    gdf.total_bounds)
    bounds.append((bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat))
    
bounds_df = pd.DataFrame(bounds).rename(
    index={0: 'Rogue River-Siskyou Bounds', 1: 'Los Padres Bounds'},
    columns={0: 'min_lon', 1: 'min_lat', 2: 'max_lon', 3: 'max_lat'})
display(bounds_df)

#  Define the Caddo download URL template for the soil data
soil_url_template = ("http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/"
            "v1.0"
            "/ph"
            "/mean"
            "/60_100"
            "/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif")
display(soil_url_template)

# Maybe turn soil_urls into a dictionary and instead of appending in line 40 add results to dictionary
# Check notebook 92 from redlining code
soil_urls = []
for site_name, site_bounds_df in bounds_df.groupby(level=0):
    print(site_name)
    for min_lon in range(floor(site_bounds_df.min_lon), ceil(site_bounds_df.max_lon)):
        for min_lat in range(floor(site_bounds_df.min_lat), ceil(site_bounds_df.max_lat)):
            print(min_lon, min_lat)
            soil_url = soil_url_template.format(
                min_lat=min_lat, max_lat=min_lat+1,
                min_lon=min_lon, max_lon=min_lon+1)
            soil_urls.append(soil_url)
display(soil_urls)

AttributeError: 'str' object has no attribute 'total_bounds'

In [ ]:
for gdf in sites_gdf.iterrows():
    print(gdf)

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [19]:
sites_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPEAREA,SHAPELEN,geometry
0,200683,295464010328,Siskiyou National Forest,1161692.775,0.513890,6.990039,"MULTIPOLYGON (((-124.02284 42.83323, -124.0209..."
1,200684,295465010328,Rogue River National Forest,680891.523,0.301755,7.115616,"MULTIPOLYGON (((-122.27322 43.13317, -122.2716..."
2,200607,295387010328,Los Padres National Forest,1956740.523,0.782005,12.961269,"MULTIPOLYGON (((-120.74838 35.44383, -120.7484..."


### Topographic data

One way to access reliable elevation data is from the [SRTM
dataset](https://www.earthdata.nasa.gov/data/instruments/srtm),
available through the [earthaccess
API](https://earthaccess.readthedocs.io/en/latest/quick-start/).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download SRTM elevation data for a particular location and
calculate any additional topographic variables you need such as slope or
aspect.</p>
<p>Then, use loops to download and organize the rasters you will need to
complete this section. Include topographic parameters that will help you
to answer your scientific question.</p></div></div>

> **Warning**
>
> Be careful when computing the slope from elevation that the units of
> elevation match the projection units (e.g. meters and meters, not
> meters and degrees). You will need to project the SRTM data to
> complete this calculation correctly.

In [3]:
# Download soil data

### Climate model data

You can use MACAv2 data for historical and future climate data. Be sure
to compare at least two 30-year time periods (e.g. historical vs. 10
years in the future) for at least four of the CMIP models. Overall, you
should be downloading at least 8 climate rasters for each of your sites,
for a total of 16. **You will *need* to use loops and/or functions to do
this cleanly!**.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download MACAv2 data for a particular climate model, emissions
scenario, spatial domain, and time frame. Then, use loops to download
and organize the 16+ rasters you will need to complete this section. The
<a
href="http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html">MACAv2
dataset is accessible from their Thredds server</a>. Include an
arrangement of sites, models, emissions scenarios, and time periods that
will help you to answer your scientific question.</p></div></div>

In [4]:
# Download climate data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Make sure to include a description of the climate data and how you
selected your models. Include a citation of the MACAv2 data</p></div></div>

YOUR CLIMATE DATA DESCRIPTION AND CITATIONS HERE